In [1]:
import jieba
import pymysql
import word2vec
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
curr_dir = '/home/souche/qiongjiu/hgc/'

In [2]:
# 首先读取车型的TF-IDF值, 以及车型名称
# 需要从数据库读取相关的数据
car_data_columns= ["model_code","model_name","value"]
tf = pd.read_csv(curr_dir+'tfidf_to_sql',header=None,sep='\t',names=car_data_columns)

car_data_columns= ["country_id","manufacturer","battery_quality_mile","battery_quality_time","brand_code","brand_name","car_body",
"cylinder_number","driving_mode","engine_volume_l","fuel_form","gear_box_type","guide_price","height","import_type",
"intake_type","length","level","max_power","max_torque","model_code","model_name","quality_mile","quality_time","rate",
"rate_count","seat_number_top","series_code","series_level","series_name","wheel_base","width","year","down_market_time",
"production_time","shutdown_time","time_to_market","year_1","year_2","year_3","year_4","year_5","year_6","year_7","year_8",
"year_9","year_10","year_11","year_12","year_13","year_14","year_15","year_16"]
model = pd.read_csv(curr_dir+'all_car_model_sep_word_to_sql',header=None,sep='\t',names=car_data_columns)
model_name = model[['model_code','model_name']]


In [3]:
tf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60741 entries, 0 to 60740
Data columns (total 3 columns):
model_code    60741 non-null object
model_name    60741 non-null object
value         60741 non-null object
dtypes: object(3)
memory usage: 1.4+ MB


In [4]:
tf['model_code'] = tf['model_code'].astype('str')
model_name['model_code'] = model_name['model_code'].astype('str')
tf = tf.rename(columns={"model_name": "model_name_list"})
data = pd.merge(tf, model_name, how='inner', on='model_code')
#data.info()

In [5]:
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60741 entries, 0 to 60740
Data columns (total 4 columns):
model_code         60741 non-null object
model_name_list    60741 non-null object
value              60741 non-null object
model_name         60741 non-null object
dtypes: object(4)
memory usage: 1.9+ MB


In [6]:
import re
file_name = "tf_idf_model_name_split_word.txt"

with open(file_name, 'w', encoding='utf-8') as f:
    for _,tar in data.iterrows():
        model_name_list = re.split(r'\s+', tar['model_name'].strip())
        for eachseg in model_name_list:
            f.write("%s " % eachseg.strip())


In [7]:
#100维 
word2vec.word2vec('tf_idf_model_name_split_word.txt', 
                  'tf_idf_model_name.bin', 
                  size = 100, 
                  window = 5, 
                  min_count = 1, 
                  verbose = True)

model_100 = word2vec.load('tf_idf_model_name.bin')
model_100.vectors.shape

Starting training using file tf_idf_model_name_split_word.txt
Vocab size: 9263
Words in train file: 529061
Alpha: 0.000002  Progress: 100.82%  Words/thread/sec: 315.95k  

(9263, 100)

In [8]:
# TF-IDF作为权重的词向量
# 将所有词的词向量加权平均后作为model_name的表示
def getmodelnameVec(modelname, tfidf):
    #seg_list = modelname.strip().split(' ')
    seg_list = re.split(r'[\s]+', modelname.strip())
    weight_list = [float(x) for x in tfidf.strip('[]').split(',')]
    resvec = np.linspace(0, 0, 100)
    #i = 0
    for i in range(len(seg_list)):
        try:
            model_100[seg_list[i].strip()]
        except Exception as err:
            print ('not found the word:', seg_list[i])
            continue
        resvec = resvec + model_100[seg_list[i].strip()] * weight_list[i]
            #i = i + 1
    return resvec/sum(weight_list)
model_name_vec = pd.DataFrame(list(map(lambda modelname, tfidf:getmodelnameVec(modelname, tfidf),data['model_name'], data['value'])),columns=['wordvec'+str(count) for count in range(0,100,1)])
data_new = pd.concat([data,model_name_vec],axis=1)

In [9]:
data_new_save = data_new.drop(['model_name_list', 'value'], axis=1)
data_new_save.head()


,model_code,model_name,wordvec0,wordvec1,wordvec2,wordvec3,wordvec4,wordvec5,wordvec6,wordvec7,...,wordvec90,wordvec91,wordvec92,wordvec93,wordvec94,wordvec95,wordvec96,wordvec97,wordvec98,wordvec99
0,43858,2012 江淮 汽车 瑞风M2 1.8 L MT 豪华 运动 5 座,0.119190,0.011224,-0.001500,0.041777,0.045343,0.003906,0.008099,-0.019319,...,-0.087833,0.063792,-0.028972,0.092506,0.022858,0.005506,0.010069,0.125314,0.013541,0.021961
1,43352,2007 东风 本田 思域 1.3 L 混合 动力,0.001615,0.051149,0.034584,0.005680,0.096276,0.066889,0.028695,-0.082298,...,0.022983,-0.016967,-0.035237,0.009152,0.021280,0.057920,-0.036720,0.099472,-0.085489,0.022000
2,117662,2011 路虎 进口 揽胜 5.0 V8 SC,-0.023109,-0.043687,-0.056806,-0.109171,-0.024376,0.004217,0.033964,0.009512,...,0.223775,0.096273,-0.066114,0.080903,-0.012162,0.055414,0.000660,0.017783,0.040829,-0.191560
3,11525-n,2016 斯巴鲁 森林人 2.0 i 时尚 导航,0.052280,-0.002271,0.133933,-0.107789,0.076205,-0.060816,0.080373,-0.027167,...,0.065572,0.098734,0.039020,0.041456,0.115441,0.028278,-0.043797,-0.021796,-0.096256,-0.099791
4,365842,2021 东风 悦达起亚 起亚K3 1.4 T DCT GTLine 奋斗,0.026885,-0.120051,0.066731,-0.161141,0.014145,-0.001033,0.061640,-0.074332,...,-0.077801,-0.007498,-0.073266,0.096668,0.023627,0.084558,-0.021886,-0.027234,0.094833,0.020672


In [10]:
data_new_save.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60741 entries, 0 to 60740
Columns: 102 entries, model_code to wordvec99
dtypes: float64(100), object(2)
memory usage: 47.3+ MB


In [11]:
# 输出 词向量的文件 
data_new_save.to_csv(curr_dir+"2021-02-02版-TF-IDF权重的词向量.csv", header=True, index=False)
